##Account name/Account ID Reconciliation

###Justin Parsons
###MSDS696 S8W2 2025

This notebook contains EDA focused on account_id and account_name. During MSDS692 it was discovered that multiple account_names are assigned to single account_ids. Multiple producers are listed under more than one name, not all of which can be accounted for by performing normal NLP treatments. For example acquisitions and company name changes appear as more than one producer if only considering account_name entries.  

Note: I found that uploading the data to the colab environment truncates the data. To remedy this I've uploaded the data to google drive and am accessing it there.

In [1]:
import pandas as pd
import numpy as np
import re

from google.colab import drive

In [2]:
drive.mount('/content/drive/')
path = "/content/drive/MyDrive/GHG/Data/"


#create dataframe
file_name = r'Mega_merged_all_real_values.csv'

#this is here as a reminder that drive connections must be set properly for personal environment
try:
  df = pd.read_csv(path+file_name)
  print(df.sample(5)) # 5 samples from the DF
except FileNotFoundError:
  print(f"Error: File not found at {file_name}")
except Exception as e:
  print(f"An error occurred: {e}")


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


<ipython-input-2-22598af36989>:11: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path+file_name)


        account_id                    account_name  \
82672        21365             ALPS ALPINE CO LTD.   
111557       16891         Shinhan Financial Group   
58841        35831  Freshfields Bruckhaus Deringer   
110551       61702                          ROSSOW   
139834      856838         PT.Anggun Kreasi Garmen   

                                     incorporated_country      ticker  \
82672                                               Japan     6770 JP   
111557                                  Republic of Korea   055550 KS   
58841   United Kingdom of Great Britain and Northern I...         NaN   
110551                                             France         NaN   
139834                                          Indonesia         NaN   

                         Evaluation_status  \
82672                 Relevant, calculated   
111557                Relevant, calculated   
58841   Not relevant, explanation provided   
110551                Relevant, calculated   
139834

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181156 entries, 0 to 181155
Data columns (total 42 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   account_id                    181156 non-null  int64  
 1   account_name                  181156 non-null  object 
 2   incorporated_country          181156 non-null  object 
 3   ticker                        117989 non-null  object 
 4   Evaluation_status             180307 non-null  object 
 5   Scope_3_emissions_type        181156 non-null  object 
 6   Scope_3_emissions_amount      181156 non-null  float64
 7   Symbol_1                      181156 non-null  object 
 8   Year                          181156 non-null  int64  
 9   Market_Cap                    115901 non-null  float64
 10  Employee_Count                23397 non-null   float64
 11  ebitda                        118470 non-null  float64
 12  Revenue                       118470 non-nul

In [4]:
#cleans account_name

def remove_whitespace_punctuation(account_name):
  if account_name is None:
    return None
  account_name = re.sub(r'\s+', '', account_name)  #remove whitespace
  account_name = re.sub(r'[^\w\s]', '', account_name)  #remove punctuation
  return account_name


In [5]:
df['account_name'] = df['account_name'].apply(remove_whitespace_punctuation)

In [6]:
#displaying unique ID and account names.
try:
  unique_account_id_count = df['account_id'].nunique()
  unique_account_name_count = df['account_name'].nunique()

  print(f"Number of unique account IDs: {unique_account_id_count}")
  print(f"Number of unique account names: {unique_account_name_count}")

except Exception as e:
  print(f"An error occurred: {e}")


Number of unique account IDs: 8795
Number of unique account names: 9319


I've decided to standardize account_name by the most recently reported account_name for each account_id.

In [7]:
#get most recent account_id/name
df['Year'] = pd.to_datetime(df['Year'], errors='coerce').dt.year  # Convert Year to a numeric year
account_id_most_recent_year = df.groupby('account_id')['Year'].max()

#dictionary that maps account_id to the most recent account_name
account_id_to_most_recent_name = {}
for account_id, most_recent_year in account_id_most_recent_year.items():
  most_recent_name = df[(df['account_id'] == account_id) & (df['Year'] == most_recent_year)]['account_name'].iloc[0]
  account_id_to_most_recent_name[account_id] = most_recent_name

#replace account_names with the most recent account_name for each account_id
df['account_name'] = df['account_id'].map(account_id_to_most_recent_name)

#verify that account names have been merged to account ID
#this number should agree with unique account IDs above
unique_account_name_count = df['account_name'].nunique()
print(f"Number of unique account names: {unique_account_name_count}")

Number of unique account names: 8792
